# Capstone 2: Obesity in America
## Data Wrangling

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### Data Collection

Checking that I am in the correct directory and change if need be. 

In [2]:
ls

Capstone2_2.ipynb               data/
Capstone_BRFSS_Obesity_CSV.csv  figures/
DataWrangling_Capstone2.ipynb   models/
ProjectProposal_Capstone2.pdf


Import my dataset using read_csv and taking a look at the first 5 rows to get an idea of what the dataframe looks like. 

In [2]:
df = pd.read_csv('Capstone_BRFSS_Obesity_CSV.csv')

#### Data Organization

Creating a file structure to store my data, figures and models I create.  

In [12]:
path = 'springboard/Capstone2Project'
print ("The current working directory is %s" % path)

The current working directory is springboard/Capstone2Project


In [13]:
mkdir data

mkdir: data: File exists


In [14]:
mkdir figures

mkdir: figures: File exists


In [15]:
mkdir models

mkdir: models: File exists


#### Data Definition

At this point, I want to gain an understanding of what my data looks like, and what might need to happen to make it cleaner to work with later on. First, I am going to drop some columns I don't need.  
-YearEnd is the same as YearStart, DataSource is the same for all observations, Data Footnote Symbol is unnecessary because we have the footnote itself, and we don't need to know confidence limits. 

In [3]:
df = df.drop(['YearEnd', 'Datasource', 'Data_Value_Unit', 'Data_Value_Footnote_Symbol', 'Data_Value_Type', 'DataValueTypeID', 'Data_Value_Alt', 'Low_Confidence_Limit', 'High_Confidence_Limit '], axis=1)

Next I am going to take a look at null values and data types. 

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53392 entries, 0 to 53391
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   YearStart                  53392 non-null  int64  
 1   LocationAbbr               53392 non-null  object 
 2   LocationDesc               53392 non-null  object 
 3   Class                      53392 non-null  object 
 4   Topic                      53392 non-null  object 
 5   Question                   53392 non-null  object 
 6   Data_Value                 48346 non-null  float64
 7   Data_Value_Footnote        5046 non-null   object 
 8   Sample_Size                48346 non-null  float64
 9   Total                      1907 non-null   object 
 10  Age(years)                 11438 non-null  object 
 11  Education                  7628 non-null   object 
 12  Gender                     3814 non-null   object 
 13  Income                     13349 non-null  obj

In [5]:
df.shape

(53392, 24)

In [6]:
df.isnull().mean()

YearStart                    0.000000
LocationAbbr                 0.000000
LocationDesc                 0.000000
Class                        0.000000
Topic                        0.000000
Question                     0.000000
Data_Value                   0.094509
Data_Value_Footnote          0.905491
Sample_Size                  0.094509
Total                        0.964283
Age(years)                   0.785773
Education                    0.857132
Gender                       0.928566
Income                       0.749981
Race/Ethnicity               0.714264
GeoLocation                  0.018879
ClassID                      0.000000
TopicID                      0.000000
QuestionID                   0.000000
LocationID                   0.000000
StratificationCategory1      0.000000
Stratification1              0.000000
StratificationCategoryId1    0.000000
StratificationID1            0.000000
dtype: float64

A high percentage of Race/Ethnicity, Income, Gender, Education, Age, and Total are null. We can delete these columns because these attributes are categorized neatly for us in the stratification columns. 

In [7]:
df = df.drop(['Race/Ethnicity', 'Income', 'Gender', 'Education', 'Age(years)'], axis=1)

In [8]:
df = df.drop(['Total'], axis = 1)

Data_Value_Footnote also has a high percentage of null values, lets look at why. 

In [9]:
df['Data_Value_Footnote'].unique()

array([nan, 'Data not available because sample size is insufficient.',
       'Data not available because sample size is insufficient.  If data only missing for the confidence interval, the confidence interval was not calculated.'],
      dtype=object)

In [10]:
pd.notnull(df['Data_Value_Footnote']).value_counts()

False    48346
True      5046
Name: Data_Value_Footnote, dtype: int64

In [11]:
pd.notnull(df['Sample_Size']).value_counts()

True     48346
False     5046
Name: Sample_Size, dtype: int64

In [12]:
# for which states do we have null values?
df_group = df.groupby(['Data_Value_Footnote', 'LocationDesc'])

It appears that the data value footnotes indicate rows where the sample size is insufficient, which accounts for 5,046 of our observations.  This also accounts for our null values in the Sample_Size column. So we will drop the rows where the sample size was insufficient

In [14]:
df = df[df['Data_Value'].notna()]

In [18]:
df = df.drop(['Data_Value_Footnote'], axis = 1)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48346 entries, 0 to 53386
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   YearStart                  48346 non-null  int64  
 1   LocationAbbr               48346 non-null  object 
 2   LocationDesc               48346 non-null  object 
 3   Class                      48346 non-null  object 
 4   Topic                      48346 non-null  object 
 5   Question                   48346 non-null  object 
 6   Data_Value                 48346 non-null  float64
 7   Sample_Size                48346 non-null  float64
 8   GeoLocation                47338 non-null  object 
 9   ClassID                    48346 non-null  object 
 10  TopicID                    48346 non-null  object 
 11  QuestionID                 48346 non-null  object 
 12  LocationID                 48346 non-null  int64  
 13  StratificationCategory1    48346 non-null  obj

### Data Cleaning

Lets rename some of our columns so they are more descriptive of what they contain. 

In [23]:
df.rename(columns={'YearStart':'Year', 'LocationDesc':'Location', 'Data_Value':'Percent'}, inplace=True)

Looking closer at the unique values in the data set, it appears there are 55 locations.  We only want to look at the 50 states, so lets identify which are not states and drop those from our data frame. 

In [24]:
df.nunique()

Year                            6
LocationAbbr                   55
Location                       55
Class                           3
Topic                           3
Question                        9
Percent                       669
Sample_Size                  8123
GeoLocation                    54
ClassID                         3
TopicID                         3
QuestionID                      9
LocationID                     55
StratificationCategory1         6
Stratification1                28
StratificationCategoryId1       6
StratificationID1              28
dtype: int64

In [25]:
df['Location'].unique()

array(['Alabama', 'National', 'Alaska', 'Arizona', 'Arkansas',
       'California', 'Connecticut', 'Colorado', 'Delaware', 'Florida',
       'District of Columbia', 'Georgia', 'Guam', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'West Virginia', 'Washington', 'Wisconsin', 'Wyoming',
       'Virgin Islands'], dtype=object)

In [26]:
non_states = ['National', 'Guam', 'District of Columbia', 'Puerto Rico', 'Virgin Islands']

In [27]:
df = df[~df['Location'].isin(non_states)]

In [30]:
df['Location'].nunique()

50

In [31]:
df.describe()

,Year,Percent,Sample_Size,LocationID
count,45629.000000,45629.000000,45629.000000,45629.000000
mean,2013.242762,31.176254,2032.622806,29.198054
std,1.690712,10.220526,2532.787103,15.622223
min,2011.000000,1.900000,50.000000,1.000000
25%,2011.000000,24.100000,578.000000,17.000000
50%,2013.000000,30.700000,1201.000000,29.000000
75%,2015.000000,37.000000,2444.000000,42.000000
max,2016.000000,77.600000,36868.000000,56.000000


In [32]:
pd.to_datetime(df['Year'], format='%Y')

0       2011-01-01
1       2011-01-01
2       2011-01-01
3       2011-01-01
4       2011-01-01
           ...    
53301   2016-01-01
53302   2016-01-01
53303   2016-01-01
53305   2016-01-01
53309   2016-01-01
Name: Year, Length: 45629, dtype: datetime64[ns]